In [2]:
import tensorflow as tf
import torch
from tensorflow.keras.models import load_model
import torch.nn.functional as F

# Load Keras models (.h5 and .keras)
model_h5 = load_model(r'C:\Users\Administrator\Desktop\FINAL YEAR PROJECT\Models\CNN_alzheimers_model (keras).h5')
model_keras = load_model(r'C:\Users\Administrator\Desktop\FINAL YEAR PROJECT\Models\deep_cnn_model.keras')

import torch
from transformers import ViTForImageClassification

# Step 1: Define the ViT model architecture with num_labels = 4
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=4,
    ignore_mismatched_sizes=True  # Ignore size mismatch for the classification layer
)

# Step 2: Load the saved state dictionary (weights)
state_dict = torch.load(r'C:\Users\Administrator\Desktop\FINAL YEAR PROJECT\Models\ViT_alzheimers_model (torch).pth', map_location=torch.device('cpu'))

# Step 3: Load the state_dict into the model
model.load_state_dict(state_dict, strict=False)  # Use strict=False if there are any further discrepancies

# Step 4: Set the model to evaluation mode
model.eval()

print("Model loaded successfully!")



C:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 64 variables whereas the saved optimizer has 126 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11324\1840944726.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the cu

Model loaded successfully!


In [15]:
from PIL import Image
import numpy as np
import torchvision.transforms as transforms

def preprocess_image(image_path, img_size):
    # Open the image using PIL and convert to RGB
    img = Image.open(image_path).convert('RGB')
    
    # Preprocess for Keras (TensorFlow expects channels-last format)
    keras_transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),  # Convert to Tensor (PyTorch channels-first format)
    ])
    img_keras = keras_transform(img).unsqueeze(0).numpy()  # Convert to NumPy array
    img_keras = np.transpose(img_keras, (0, 2, 3, 1))  # Reorder to channels-last (batch_size, height, width, channels)
    
    # Preprocess for PyTorch (channels-first format)
    torch_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(img_size),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img_pth = torch_transform(img).unsqueeze(0)  # Add batch dimension for PyTorch
    
    return img_keras, img_pth


In [20]:
import torch
import torch.nn.functional as F

# Assuming you already have the path to your PyTorch model
pth_model_path = r'C:\Users\Administrator\Desktop\FINAL YEAR PROJECT\Models\ViT_alzheimers_model (torch).pth'

# Load the PyTorch model
model_pth = torch.load(pth_model_path)
model_pth.eval()  # Set the model to evaluation mode

def get_predictions(image_path):
    # Preprocess the image
    img_keras, img_pth = preprocess_image(image_path, (128, 128))
    
    # Predict with Keras models
    pred_h5 = model_h5.predict(img_keras)
    pred_keras = model_keras.predict(img_keras)
    
    # Predict with PyTorch model
    with torch.no_grad():
        pred_pth = model_pth(img_pth)
        pred_pth = F.softmax(pred_pth, dim=1).numpy()  # Get probabilities
    
    return pred_h5, pred_keras, pred_pth


C:\Users\Administrator\AppData\Local\Temp\ipykernel_11324\2282322604.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_pth = torch.load(pth_model_path)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [17]:
def combine_predictions(pred_h5, pred_keras, pred_pth):
    # Average the predictions from the three models
    avg_pred = (pred_h5 + pred_keras + pred_pth) / 3.0
    
    # Get the class with the highest average probability
    final_pred = np.argmax(avg_pred, axis=1)
    
    return final_pred


In [18]:
def evaluate_accuracy(test_image_paths, true_labels):
    correct = 0
    total = len(test_image_paths)

    for i, image_path in enumerate(test_image_paths):
        # Get predictions from each model
        pred_h5, pred_keras, pred_pth = get_predictions(image_path)

        # Combine the predictions and get final prediction
        final_pred = combine_predictions(pred_h5, pred_keras, pred_pth)

        # Compare with true label
        if final_pred == true_labels[i]:
            correct += 1

    accuracy = (correct / total) * 100
    print(f"Ensemble Model Accuracy: {accuracy:.2f}%")


In [19]:
test_images = [r'C:\Users\Administrator\Desktop\FINAL YEAR PROJECT\Dataset\Sample test.jpg']
true_labels = [0, 1, 2, ...]  # True labels for the test set

evaluate_accuracy(test_images, true_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 22s 22s/step


NameError: name 'model_pth' is not defined